**1. Mount drive and install libararies**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2. Import libraries**

In [ ]:
!pip install torch torchvision
!pip install transformers==2.10.0
!pip install seqeval
!pip install tensorboardx
!pip install simpletransformers==0.9.1

import pandas as pd
import numpy as np

import gc
import requests
import random 
import os

from sklearn.model_selection import KFold
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score
from scipy.special import softmax

import torch

print("Cuda available" if torch.cuda.is_available() is True else "CPU")
print("PyTorch version: ", torch.__version__)

**3. Download the data into PD DataFrame**

In [ ]:
# read base training set (two coders)

path = '/content/drive/MyDrive/spap_state/spap_state_vaccine/classification/' # set own path
df_random_with_non_relevant = pd.read_csv(path + 'state_twitter_vaccine_hand_label_9200_19_april.csv', encoding = 'latin1') # this contains tweets unrelated to vaccine
df_random_with_non_relevant.rename({'full_text_final_entire': 'full_text_final'}, # column name change
                                   axis = 1, 
                                   inplace = True)

df_random_with_non_relevant = df_random_with_non_relevant[df_random_with_non_relevant['full_text_final'].isna()==False] # remove junk rows where text cannot be read

df_random = df_random_with_non_relevant[df_random_with_non_relevant['stance_final'] != 9] # remove tweets unrelated to vaccine

print(df_random['stance_final'].value_counts()) # three-fold labels

df_random['stance_final_anti'] = np.where(df_random['stance_final'] == -1, 1, 0) # anti-vax is re-labeled as 1 while rest as 0
print(df_random['stance_final_anti'].value_counts()) # two-fold labels (anti vs. rest)

df_random = df_random[['full_text_final', 'stance_final_anti']] # subset only relevant columns
df_random['label_anti'] = df_random['stance_final_anti'].astype(float)
df_random = df_random[['full_text_final','label_anti']]

# read training set from active learning (one coder)

df_active_all = []
round = 23

for i in range(1, round + 1):
  df_active_with_non_relevant = pd.read_csv(path + 'active_learning_round_' + str(i) + '.csv', encoding = 'latin1')
  df_active_with_non_relevant['full_text_final'] = df_active_with_non_relevant['full_text'] 
  df_active = df_active_with_non_relevant[df_active_with_non_relevant['stance_final'] != 9]
  df_active['label_anti'] = np.where(df_active['stance_final'] == -1, 1, 0)
  df_active = df_active[['full_text_final', 'label_anti']]
  df_active_all.append(df_active)

df_active_all = pd.concat(df_active_all)

# join two training sets

df = pd.concat([df_random, df_active_all])
df['full_text_alphabet'] = df.full_text_final.str.replace('[^a-zA-Z]', '')
df = df.drop_duplicates(subset = 'full_text_alphabet', keep = 'first', inplace = False, ignore_index = False) # drop duplicates
del df['full_text_alphabet']
df = df.sample(frac = 1).reset_index(drop = True) # shuffle

print(len(df['label_anti']))
print(df['label_anti'].value_counts(normalize=True))
print(df['label_anti'].value_counts(normalize=False))

**4. Performance metrics**

In [6]:
def report_results(A, B):
  A_name = A.name
  B_name = B.name
    
  df = pd.DataFrame({'A':A,
                       'B':B})
  df = df.dropna()
  A = df['A']
  B = df['B']
    
  acc = accuracy_score(B, A)
  f1 = f1_score(B, A)
  prec = precision_score(B, A)
  rec = recall_score(B, A)
  ROC = roc_auc_score(B, A)
    
  print('Candidate: '+A_name+' | Ground Truth: '+B_name+'\n')
  print('accuracy: %0.2f \nprecision: %0.2f \nrecall: %0.2f \nF1 score: %0.2f \nROC AUC: %0.2f \n' % (acc, prec, rec, f1, ROC))

  performance = [prec, rec, f1, ROC, acc]

  return performance

**5. Set seed for deterministic modeling**

In [9]:
def set_seed(seed):
  
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  
  np.random.seed(seed)
  
  random.seed(seed)
  
  os.environ['PYTHONHASHSEED'] = str(seed)

**6. Base model with CV (K=5)**

In [ ]:
for learning_rate in [3e-5, 4e-5, 5e-5]:

  for num_train_epochs in [3, 4, 5]:

    print('\n\nWorking on', learning_rate, '+', num_train_epochs)

    args = {
      'output_dir': 'outputs/',
      'cache_dir': 'cache/',
      'fp16': False,
      'fp16_opt_level': 'O1',
      'max_seq_length': 250,
      'train_batch_size': 8,
      'eval_batch_size': 8,
      'gradient_accumulation_steps': 1,
      'num_train_epochs': num_train_epochs,
      'weight_decay': 0,
      'learning_rate': learning_rate,
      'adam_epsilon': 1e-8,
      'warmup_ratio': 0.06,
      'warmup_steps': 0,
      'max_grad_norm': 1.0,
      'logging_steps': 50,
      'evaluate_during_training': False,
      'save_steps': 2000,
      'eval_all_checkpoints': True,
      'use_tensorboard': True,
      'overwrite_output_dir': True,
      'reprocess_input_data': True}

    set_seed = 888
    n = 5
    kf = KFold(n_splits = n, random_state = 888, shuffle = True)

    performance_cv = []
    for train_index, val_index in kf.split(df):
      # splitting Dataframe (dataset not included)
        train_df = df.iloc[train_index]
        val_df = df.iloc[val_index]
      # Defining Model
        model = ClassificationModel('bert', 'bert-base-uncased', args = args)
      # train the model
        model.train_model(train_df)
      # validate the model 
        result, model_outputs, wrong_predictions = model.eval_model(val_df)
        val_df['BERT_label'] = np.argmax(model_outputs, axis = 1)
      # performance
        performance = report_results(val_df['BERT_label'], val_df['label_anti'])
        performance_cv.append(performance)

    df_performance = pd.DataFrame(performance_cv)
    df_performance.columns = ['prec', 'rec', 'f1', 'ROC', 'acc']

    print('mean', df_performance['prec'].mean(), df_performance['rec'].mean())
    print('sd', df_performance['prec'].std(), df_performance['rec'].std(), '\n\n\n\n')



Working on 3e-05 + 3
Converting to features started.


  0%|          | 0/3600 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Current iteration:   0%|          | 0/450 [00:00<?, ?it/s]

Running loss: 0.298483

...

Running loss: 0.567827

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.347673

Current iteration:   0%|          | 0/450 [00:00<?, ?it/s]

Running loss: 0.269326